# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [5]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

## 1. Ten Countries with the Lowest Infant Mortality Rates

In [6]:
document = ET.parse( './data/mondial_database.xml' )

def read_ethnic_groups(country_element):
    ethnic_groups = []
    for ethnic_group_elem in country_element.getiterator('ethnicgroup'):
        percent = float(ethnic_group_elem.attrib['percentage']) / 100
        name = ethnic_group_elem.text
        ethnic_groups.append({'name' : name, 'percentage': percent})
    return ethnic_groups

# Reader converts XML to Python list
def read_countries(document):
    countries = []
    for element in document.iterfind('country'):
        name = element.find('name').text
        pop_elem = element.find('population')
        country_pop = int(pop_elem.text)
        country_pop_year = pop_elem.attrib['year']        
        ethnic_groups = read_ethnic_groups(element)
        infant_mortality = element.find('infant_mortality')
        cities = []
        for city_element in element.getiterator('city'):
            city = {'name' : city_element.find('name').text}
            for population_element in city_element.getiterator('population'):
                population_year = population_element.attrib['year']
                population_num = int(population_element.text)
                city['year']= population_year
                city['population'] = population_num
            cities.append(city)
        countries.append({'name' : name,
                          'population' : country_pop,
                          'year' : country_pop_year,
                          'cities' : cities,
                          'infant_mortality' : infant_mortality.text if infant_mortality is not None else None,
                          'ethnic_groups' : ethnic_groups
                         })
    return countries

document = ET.parse( './data/mondial_database.xml' )
countries = read_countries(document)

import pandas as pd
countries_df = pd.DataFrame(countries)
countries_df.dropna(inplace=True)
countries_df['infant_mortality'] = pd.to_numeric(countries_df['infant_mortality'])
countries_df.nsmallest(10, 'infant_mortality')[['name', 'infant_mortality']]


,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
36,Norway,2.48
117,Bermuda,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


## 2. Ten Cities with the Largest Population

In [7]:
from pandas.io.json import json_normalize

df = pd.DataFrame()
for country in countries:
    city_df = json_normalize(country, 'cities') # Normalize JSON
    city_df['country'] = country['name']
    df = df.append(city_df)

# Drop rows with missing values
df = df.dropna(axis=0)

# Use the maximum value of the census count for each city as the
# basis for comparison, since some of the values for population are missing.
# This assumes a positive trend in population, which should be true for most large cities.
# We could also use interpolation to replace the missing values (see section below).
max_populations_by_city = df.groupby('name')['population'].max()

cities = max_populations_by_city.index.values
population = max_populations_by_city.values
population
cities_df = pd.DataFrame({'city' : cities, 
                          'population' : population})
cities_df.sort_values(by='population', 
                      ascending=False)[:10]


,city,population
2380,Shanghai,22315474.0
1127,Istanbul,13710512.0
1742,Mumbai,12442373.0
1729,Moskva,11979529.0
283,Beijing,11716620.0
2535,São Paulo,11152344.0
2631,Tianjin,11090314.0
941,Guangzhou,11071424.0
686,Delhi,11034555.0
2393,Shenzhen,10358381.0


## Top 10 Cities by Population: With Interpolation of missing values

The results are the same whether we use interpolation or use the maximum population for each city:

In [8]:
df = pd.DataFrame()
for country in countries:
    city_df = json_normalize(country, 'cities') # Normalize JSON
    city_df['country'] = country['name']
    df = df.append(city_df)

# Remove rows with missing years
df = df.dropna(subset=['year'])
df = df.set_index(['name', 'year'])

df=df.interpolate(method='linear')
df.sort_values(by='population', 
               ascending=False)[:10]

,,country,population
name,year,,
Shanghai,2010,China,22315474.0
Istanbul,2012,Turkey,13710512.0
Mumbai,2011,India,12442373.0
Moskva,2013,Russia,11979529.0
Beijing,2010,China,11716620.0
São Paulo,2010,Brazil,11152344.0
Tianjin,2010,China,11090314.0
Guangzhou,2010,China,11071424.0
Delhi,2011,India,11034555.0


## 3. Top 10 Ethnic Groups


In [9]:
df = pd.DataFrame()
for country in countries:
    eg_df = json_normalize(country, 'ethnic_groups') # Normalize JSON
    eg_df['country'] = country['name']
    eg_df['population'] = country['population']
    eg_df['year'] = country['year']
    df = df.append(eg_df)

# Remove missing years
df = df.dropna(subset=['year'])

def get_top_10_ethnic_groups(df):
    df['ethnic_est_pop'] = df['population'] * df['percentage']
    pop_counts = df.groupby('name').sum()
    pop_counts = pop_counts.drop(['population', 'percentage'], axis=1)
    return pop_counts.sort_values(by='ethnic_est_pop', ascending=False)[:10]

get_top_10_ethnic_groups(df)

,ethnic_est_pop
name,
Han Chinese,4.975551e+08
European,1.928658e+08
Indo-Aryan,1.716454e+08
Russian,9.275844e+07
African,8.632937e+07
Japanese,8.170627e+07
German,6.623219e+07
Dravidian,5.959908e+07
English,4.231499e+07


## 4.a. Longest River

In [78]:
# Country names
def find_country_name(document_root, country_code):
    country = document_root.find("country[@car_code='" + country_code + "']")
    return country.find('name').text

def get_country_names(country_codes):
    return [find_country_name(document, country_code) for country_code in country_codes]

def read_rivers(document_root):
    rivers = []
    for river_elem in document_root.iterfind('river'):        
        river = river_elem.attrib['id']
        length = river_elem.find('length')
        country = river_elem.attrib['country']
        river = {'river' : river, 
                 'length' : float(length.text) if length is not None else None,
                 'country' : country}
        rivers.append(river)
    return rivers


document = ET.parse( './data/mondial_database.xml' )
rivers = read_rivers(document)

df = pd.DataFrame(rivers)
df = df.sort_values(by='length', ascending=False)
df[:1]

,country,length,river
174,CO BR PE,6448.0,river-Amazonas


In [79]:
get_country_names(df[:1]['country'].values[0].split())

['Colombia', 'Brazil', 'Peru']

## 4b. Lake with the Largest Area

In [80]:
document = ET.parse( './data/mondial_database.xml' )

def read_lakes(document_root):
    lakes = []
    for lake_elem in document.iter('lake'):
        lake_id = lake_elem.attrib['id'] 
        area = lake_elem.find('area')
        country = lake_elem.attrib['country']
        lake = {'lake' : lake_id,
                'area' : float(area.text) if area is not None else None,
                'country' : country}
        lakes.append(lake)
    return lakes

lakes = read_lakes(document)
df = pd.DataFrame(lakes)
df = df.sort_values(by='area', ascending=False)
df[:1]    
    

,area,country,lake
54,386400.0,R AZ KAZ IR TM,lake-KaspischesMeer


In [81]:
get_country_names(df[:1]['country'].values[0].split())

['Russia', 'Azerbaijan', 'Kazakhstan', 'Iran', 'Turkmenistan']

## 4c. Airport at Highest Elevation

In [82]:
def read_airports(document_root):
    airports = []
    for airport_elem in document.iter('airport'):
        airport_code = airport_elem.attrib['iatacode']
        elev = airport_elem.find('elevation')
        country = airport_elem.attrib['country']
        elevation = float(elev.text) if elev.text is not None else None
        airport = {'airport' : airport_code,
                   'elevation' : elevation,
                   'country' : country}
        airports.append(airport)
    return airports

airports = read_airports(document)
df = pd.DataFrame(airports)
df = df.sort_values(by='elevation', ascending=False)
df[:1]

,airport,country,elevation
80,LPB,BOL,4063.0


In [83]:
get_country_names(df[:1]['country'].values[0].split())

['Bolivia']